In [ ]:
from fenics import *
from mshr import *
import matplotlib.pyplot as plt
import numpy as np

T = 10.0            # final time
num_steps = 100     # number of time steps
time = 0.0
theta = 0.5
dt = T / num_steps # time step size

N = 20 # resolution
R = 0.1  # brain circle Radius
ventricle_radius = R/3
aqueduct_width = R/5

kappa = 1e-2
E = 1500.0
nu = 0.49
lmbda = nu*E/((1.0-2.0*nu)*(1.0+nu))
mu = E/(2.0*(1.0+nu))

s = 1
alpha = 0.3

a_0 = 0.6
a_1 = 1.5
f_0 = 1.0/(60/15)
f_1 = 1.0/(60/70)

mmHg2Pa = 133.32


# create mesh

brain = Circle(Point(0,0), R)
ventricle = Circle(Point(0,0), ventricle_radius)
aqueduct = Rectangle(Point(-aqueduct_width/2, -R), Point(aqueduct_width/2, 0))
brain = brain - ventricle - aqueduct
mesh = generate_mesh(brain, N)

In [ ]:
u_degree = 2
p_degree = 1
V = VectorElement("CG", mesh.ufl_cell(), u_degree)
W = FiniteElement("CG", mesh.ufl_cell(), p_degree)
M = MixedElement([V, W, W])

VW = FunctionSpace(mesh, M)

up = Function(VW)
up_ = Function(VW)
u, p_T, p = split(up)
u_, p_T_, p_ = up_.split()
vq = TestFunctions(VW)
v, q_T, q = vq[0], vq[1], vq[2]

dtdp = theta*p + (1.0 - theta)*p_
dtdp_T = theta*p_T + (1.0 - theta)*p_T_

f = Constant([0.0, 0.0])
g = Constant(0.0)

def eps(u):
    return 0.5*(nabla_grad(u) + nabla_grad(u).T)

F1 = 2*mu*inner(eps(u), eps(v))*dx - p_T*div(v)*dx - inner(f,v)*dx

F2 = (s* dtdp + alpha/lmbda *(dtdp_T + dtdp) )*q *dx + kappa *inner(grad(p), grad(q))*dx - g*q*dx  

F3 = (lmbda*div(u) + p + p_T) *q_T *dx

F = F1 + F2 + F3

# boundary conditions

ventricle = CompiledSubDomain("on_boundary && (x[0]*x[0] + x[1]*x[1] < R*R*0.99)", R = R)
skull = CompiledSubDomain("on_boundary && (x[0]*x[0] + x[1]*x[1] >= R*R*0.99 )", R = R)

p_D_skull = Expression("7*mmHg2Pa*L*(a_0 * sin(2 *pi * f_0*t) + a_1 *sin(2*pi*f_1*t))",
                       a_0=a_0, a_1=a_0, f_0=f_0, f_1=f_1, t = 0,
                       mmHg2Pa=mmHg2Pa, L=(R - ventricle_radius),
                       degree = 2)


#bc_u_ventricle = DirichletBC(VW.sub(0), (0.0,0.0), ventricle, check_midpoint=False)
bc_u_skull = DirichletBC(VW.sub(0), (0.0,0.0), skull, check_midpoint=False)

bc_p_ventricle = DirichletBC(VW.sub(2), 0.0, ventricle, check_midpoint=False)
bc_p_skull = DirichletBC(VW.sub(2), p_D_skull, skull, check_midpoint=False)


bcs = [bc_u_skull, bc_p_skull,  bc_p_ventricle]

# set initial conditions

u_.assign(interpolate(Constant((0.0,0.0)), VW.sub(0).collapse()))
p_T_.assign(interpolate(Constant(0.0), VW.sub(1).collapse()))
p_.assign(interpolate(Constant(0.0), VW.sub(2).collapse()))


## output file

xdmf_file = XDMFFile("../results/biot_flow.xdmf")
xdmf_file.parameters["flush_output"] = True
xdmf_file.parameters["functions_share_mesh"] = True
xdmf_file.parameters["rewrite_function_mesh"] = False

probe_points = [Point(ventricle_radius, 0), 
                Point(R, 0), 
                Point((R + ventricle_radius)/2, 0), 
               ]

p_data = np.ndarray((num_steps, len(probe_points)))

In [ ]:
for n in range(num_steps):

    # Update current time
    time += dt
    p_D_skull.t = time

    # Compute solution
    solve(F==0, up, bcs)

    # Update previous solution
    up_.assign(up)
    
    # write to file
    u_out, p_T_out, p_out = up.split()
    
    u_out.rename("u","displacement")
    p_T_out.rename("p_total","total pressure")
    p_out.rename("p_fluid"," fluid pressure")

    xdmf_file.write(u_out, time)
    xdmf_file.write(p_T_out, time)
    xdmf_file.write(p_out, time)
    for i, point in enumerate(probe_points):
        
        p_data[n,i] = p_out(point)
        
xdmf_file.close()

In [ ]:
plt.plot(np.linspace(0,T, num_steps), p_data)
plt.xlabel("t in s")
plt.ylabel("p in Pa")
#plt.legend( [str(p.array()[:2]) for p in probe_points])

In [ ]:
plot(mesh)
for point in probe_points:
    plt.scatter(point.array()[0], point.array()[1], marker=".", s= 300, c="red")